In [1]:
'''
dependency:

anyascii                  0.3.2
anyio                     4.8.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.17.0
beautifulsoup4            4.13.1
bleach                    6.2.0
certifi                   2025.1.33
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
comm                      0.2.2
contractions              0.1.73
debugpy                   1.8.12
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.2.0
fastjsonschema            2.21.1
filelock                  3.17.0
fqdn                      1.5.1
fsspec                    2025.2.0
gensim                    4.3.3
h11                       0.14.0
httpcore                  1.0.7
httpx                     0.28.1
idna                      3.10
ipykernel                 6.29.5
ipython                   8.32.0
ipywidgets                8.1.5
isoduration               20.11.0
jedi                      0.19.2
Jinja2                    3.1.5
joblib                    1.4.2
json5                     0.10.0
jsonpointer               3.0.0
jsonschema                4.23.0
jsonschema-specifications 2024.10.1
jupyter                   1.1.1
jupyter_client            8.6.3
jupyter-console           6.6.3
jupyter_core              5.7.2
jupyter-events            0.12.0
jupyter-lsp               2.2.5
jupyter_server            2.15.0
jupyter_server_terminals  0.5.3
jupyterlab                4.3.5
jupyterlab_pygments       0.3.0
jupyterlab_server         2.27.3
jupyterlab_widgets        3.0.13
MarkupSafe                3.0.2
matplotlib-inline         0.1.7
mistune                   3.1.1
mpmath                    1.3.0
nbclient                  0.10.2
nbconvert                 7.16.6
nbformat                  5.10.4
nest-asyncio              1.6.0
networkx                  3.4.2
nltk                      3.9.1
notebook                  7.3.2
notebook_shim             0.2.4
numpy                     1.23.5
overrides                 7.7.0
packaging                 24.2
pandas                    2.2.3
pandocfilters             1.5.1
parso                     0.8.4
pexpect                   4.9.0
pillow                    11.1.0
pip                       23.1.2
platformdirs              4.3.6
prometheus_client         0.21.1
prompt_toolkit            3.0.50
psutil                    6.1.1
ptyprocess                0.7.0
pure_eval                 0.2.3
pyahocorasick             2.1.0
pycparser                 2.22
Pygments                  2.19.1
python-dateutil           2.9.0.post0
python-json-logger        3.2.1
pytz                      2025.1
PyYAML                    6.0.2
pyzmq                     26.2.1
referencing               0.36.2
regex                     2024.11.6
requests                  2.32.3
rfc3339-validator         0.1.4
rfc3986-validator         0.1.1
rpds-py                   0.22.3
scikit-learn              1.6.1
scipy                     1.13.1
Send2Trash                1.8.3
setuptools                65.5.0
six                       1.17.0
smart-open                7.1.0
sniffio                   1.3.1
soupsieve                 2.6
stack-data                0.6.3
sympy                     1.13.1
terminado                 0.18.1
textsearch                0.0.24
threadpoolctl             3.5.0
tinycss2                  1.4.0
torch                     2.6.0
torchaudio                2.6.0
torchvision               0.21.0
tornado                   6.4.2
tqdm                      4.67.1
traitlets                 5.14.3
types-python-dateutil     2.9.0.20241206
typing_extensions         4.12.2
tzdata                    2025.1
uri-template              1.3.0
urllib3                   2.3.0
wcwidth                   0.2.13
webcolors                 24.11.1
webencodings              0.5.1
websocket-client          1.8.0
widgetsnbextension        4.0.13
wrapt                     1.17.2

'''



import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from bs4 import BeautifulSoup
import re
import contractions
import ssl

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')
nltk.download('stopwords')
word2vec_model = api.load("word2vec-google-news-300")

[nltk_data] Downloading package punkt to /Users/liuming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liuming/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset Generation

In [2]:
# Load and preprocess the data
file_path = '../data/amazon_reviews_us_Office_Products_v1_00.tsv'
data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'], low_memory=False)

data['star_rating'] = pd.to_numeric(data['star_rating'], errors='coerce')
data.dropna(subset=['star_rating', 'review_body'], inplace=True)
data = data[data['star_rating'].isin([1, 2, 3, 4, 5])]
data['sentiment'] = data['star_rating'].apply(lambda x: 1 if x > 3 else (0 if x <= 2 else 2))
data.dropna(subset=['sentiment'], inplace=True)

balanced_data = pd.concat([
    data[data['star_rating'] == 1].sample(n=50000, random_state=6),
    data[data['star_rating'] == 2].sample(n=50000, random_state=6),
    data[data['star_rating'] == 3].sample(n=50000, random_state=6),
    data[data['star_rating'] == 4].sample(n=50000, random_state=6),
    data[data['star_rating'] == 5].sample(n=50000, random_state=6)
])

#pre-processing
balanced_data['token'] = balanced_data['review_body'].str.lower()

balanced_data['token'] = balanced_data['token'].astype(str).apply(lambda x: BeautifulSoup(x, "html.parser").get_text() if "<" in x or ">" in x else x)

balanced_data['token'] = balanced_data['token'].apply(lambda x: re.sub(r"http\S+|www\S+", "", x))

balanced_data['token'] = balanced_data['token'].apply(lambda x: re.sub(r"[^a-zA-Z\s]", "", x))

balanced_data['token'] = balanced_data['token'].apply(lambda x: re.sub(r"\s+", " ", x).strip())

balanced_data['token'] = balanced_data['token'].apply(contractions.fix)

stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    tokens = text.split(" ")  
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  
    return tokens

balanced_data['token'] = balanced_data['token'].apply(remove_stop_words)

# Word Embedding

In [3]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
example_words = [
    ('king', 'man', 'woman', 'queen'),
    ('excellent', 'outstanding')
]

for words in example_words:
    if len(words) == 4:
        result = word2vec_model[words[0]] - word2vec_model[words[1]] + word2vec_model[words[2]]
        # similar_word = word2vec_model.similar_by_vector(result, topn=1)[0]
        similarity = cosine_similarity(result, word2vec_model[words[3]])
        print(f"{words[0]} - {words[1]} + {words[2]} = {words[3]} {similarity}")
    else:
        similarity = word2vec_model.similarity(words[0], words[1])
        print(f"Similarity between '{words[0]}' and '{words[1]}': {similarity}")


custom_word2vec = Word2Vec(
    sentences=balanced_data['token'],
    vector_size=300,  
    window=11,        
    min_count=10,     
    workers=4         
)

for words in example_words:
    if len(words) == 4:
        result = custom_word2vec.wv[words[0]] - custom_word2vec.wv[words[1]] + custom_word2vec.wv[words[2]]
        similarity = cosine_similarity(result,custom_word2vec.wv[words[3]])
        print(f"{words[0]} - {words[1]} + {words[2]} = {words[3]} {similarity}")
    else:
        similarity = custom_word2vec.wv.similarity(words[0], words[1])
        print(f"Similarity between '{words[0]}' and '{words[1]}': {similarity}")



king - man + woman = queen 0.7300516963005066
Similarity between 'excellent' and 'outstanding': 0.556748628616333
king - man + woman = queen 0.1250358670949936
Similarity between 'excellent' and 'outstanding': 0.7751966714859009


# Simple Models

In [4]:
def compute_average_word2vec_custom(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)
def compute_average_word2vec_google(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300) 

balanced_data['w2v_google'] = balanced_data['token'].apply(lambda x: compute_average_word2vec_google(x, word2vec_model))

balanced_data['w2v_custom'] = balanced_data['token'].apply(lambda x: compute_average_word2vec_custom(x, custom_word2vec))

vectorizer = TfidfVectorizer(max_features=300)
tfidf_features = vectorizer.fit_transform(balanced_data['review_body']).toarray()


binary_data = balanced_data[balanced_data['sentiment'].isin([0, 1])]


X_train_google, X_test_google, y_train, y_test = train_test_split(
    np.vstack(binary_data['w2v_google']), binary_data['sentiment'], test_size=0.2, random_state=6
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    np.vstack(binary_data['w2v_custom']), binary_data['sentiment'], test_size=0.2, random_state=6
)

X_train_tfidf, X_test_tfidf, _, _ = train_test_split(
    tfidf_features[:len(binary_data)], binary_data['sentiment'], test_size=0.2, random_state=6
)

perceptron_google = Perceptron()
perceptron_google.fit(X_train_google, y_train)

perceptron_custom = Perceptron()
perceptron_custom.fit(X_train_custom, y_train)

perceptron_tfidf = Perceptron()
perceptron_tfidf.fit(X_train_tfidf, y_train)

y_pred_google = perceptron_google.predict(X_test_google)
y_pred_custom = perceptron_custom.predict(X_test_custom)
y_pred_tfidf = perceptron_tfidf.predict(X_test_tfidf)

acc_perceptron_google = accuracy_score(y_test, y_pred_google)
acc_perceptron_custom = accuracy_score(y_test, y_pred_custom)
acc_perceptron_tfidf = accuracy_score(y_test, y_pred_tfidf)

print(f"Perceptron Accuracy (Google Word2Vec): {acc_perceptron_google:.4f}")
print(f"Perceptron Accuracy (Custom Word2Vec): {acc_perceptron_custom:.4f}")
print(f"Perceptron Accuracy (TF-IDF): {acc_perceptron_tfidf:.4f}")

svm_google = SVC(kernel='linear')
svm_google.fit(X_train_google, y_train)

svm_custom = SVC(kernel='linear')
svm_custom.fit(X_train_custom, y_train)

svm_tfidf = SVC(kernel='linear')
svm_tfidf.fit(X_train_tfidf, y_train)

y_pred_svm_google = svm_google.predict(X_test_google)
y_pred_svm_custom = svm_custom.predict(X_test_custom)
y_pred_svm_tfidf = svm_tfidf.predict(X_test_tfidf)

acc_svm_google = accuracy_score(y_test, y_pred_svm_google)
acc_svm_custom = accuracy_score(y_test, y_pred_svm_custom)
acc_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf)

print(f"SVM Accuracy (Google Word2Vec): {acc_svm_google:.4f}")
print(f"SVM Accuracy (Custom Word2Vec): {acc_svm_custom:.4f}")
print(f"SVM Accuracy (TF-IDF): {acc_svm_tfidf:.4f}")

Perceptron Accuracy (Google Word2Vec): 0.7630
Perceptron Accuracy (Custom Word2Vec): 0.7970
Perceptron Accuracy (TF-IDF): 0.4640
SVM Accuracy (Google Word2Vec): 0.7880
SVM Accuracy (Custom Word2Vec): 0.8490
SVM Accuracy (TF-IDF): 0.4670


# Predefined Dataset & Train Function

In [5]:
class ReviewDataset(Dataset):
    '''
    self.features = tensor([[0.2314, 0.1234, 0.4321, ..., 0.0456, 0.9874, 0.7890],
        [0.0987, 0.2314, 0.2345, ..., 0.2314, 0.4567, 0.2314],
        ...
       ])
    self.labels = tensor([1, 0, 1, 1, 0, 0, 1, ...])
    '''
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

def train_model(model, train_loader, test_loader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
        
        train_acc = correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Train Accuracy: {train_acc:.4f}")
        
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
    
    test_acc = correct / total
    print(f"Test Accuracy: {test_acc:.4f}")

    return model
    
positive_reviews = balanced_data[balanced_data['sentiment'] == 0]
negative_reviews = balanced_data[balanced_data['sentiment'] == 1]
neutral_reviews = balanced_data[balanced_data['sentiment'] == 2]

all_reviews = pd.concat([positive_reviews, negative_reviews], ignore_index=True)
all_reviews_tri = pd.concat([positive_reviews, negative_reviews, neutral_reviews], ignore_index=True)

# Feedforward Neural Networks

In [6]:


class SentimentClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        '''
        (batch_size,n_featrue) -> (batch_size,128)
        (batch_size,128) -> (batch_size,64)
        (batch_size,64) -> (batch_size,2)
        (batch_size,2) -> (batch_size,2) softmax
        '''
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10, num_classes)
        self.softmax = nn.Softmax(dim=1)
        self.weight_init()

    def weight_init(self):
        nn.init.kaiming_uniform_(self.fc1.weight)
        nn.init.kaiming_uniform_(self.fc2.weight)
        nn.init.kaiming_uniform_(self.fc3.weight)
        
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.fc3.bias)

        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return self.softmax(x)

def compute_average_word2vec_custom(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

def compute_average_word2vec_google(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)


all_reviews['w2v_google_mean'] = all_reviews['token'].apply(lambda x: compute_average_word2vec_google(x, word2vec_model))
all_reviews['w2v_custom_mean'] = all_reviews['token'].apply(lambda x: compute_average_word2vec_custom(x, custom_word2vec))

all_reviews_tri['w2v_google_mean'] = all_reviews_tri['token'].apply(lambda x: compute_average_word2vec_google(x, word2vec_model))
all_reviews_tri['w2v_custom_mean'] = all_reviews_tri['token'].apply(lambda x: compute_average_word2vec_custom(x, custom_word2vec))

def compute_concatenated_word2vec_custom(tokens, model, max_len=10):
    vectors = [model.wv[word] for word in tokens if word in model.wv][:max_len]
    while len(vectors) < max_len:
        vectors.append(np.zeros(model.vector_size))  
    return np.concatenate(vectors)
def compute_concatenated_word2vec_google(tokens, model, max_len=10):
    vectors = [model[word] for word in tokens if word in model][:max_len]
    while len(vectors) < max_len:
        vectors.append(np.zeros(model.vector_size))  
    return np.concatenate(vectors)

all_reviews['w2v_google_concat'] = all_reviews['token'].apply(lambda x: compute_concatenated_word2vec_google(x, word2vec_model))
all_reviews['w2v_custom_concat'] = all_reviews['token'].apply(lambda x: compute_concatenated_word2vec_custom(x, custom_word2vec))

all_reviews_tri['w2v_google_concat'] = all_reviews_tri['token'].apply(lambda x: compute_concatenated_word2vec_google(x, word2vec_model))
all_reviews_tri['w2v_custom_concat'] = all_reviews_tri['token'].apply(lambda x: compute_concatenated_word2vec_custom(x, custom_word2vec))


features_google_mean = np.vstack(all_reviews['w2v_google_mean'])
features_custom_mean = np.vstack(all_reviews['w2v_custom_mean'])
features_google_concat = np.vstack(all_reviews['w2v_google_concat'])
features_custom_concat = np.vstack(all_reviews['w2v_custom_concat'])

labels = all_reviews['sentiment'].values


features_google_mean_tri = np.vstack(all_reviews_tri['w2v_google_mean'])
features_custom_mean_tri = np.vstack(all_reviews_tri['w2v_custom_mean'])
features_google_concat_tri = np.vstack(all_reviews_tri['w2v_google_concat'])
features_custom_concat_tri = np.vstack(all_reviews_tri['w2v_custom_concat'])

labels_tri = all_reviews_tri['sentiment'].values

X_train_google_mean, X_test_google_mean, y_train, y_test = train_test_split(features_google_mean, labels, test_size=0.2, random_state=6)
X_train_custom_mean, X_test_custom_mean, _, _ = train_test_split(features_custom_mean, labels, test_size=0.2, random_state=6)
X_train_google_concat, X_test_google_concat, _, _ = train_test_split(features_google_concat, labels, test_size=0.2, random_state=6)
X_train_custom_concat, X_test_custom_concat, _, _ = train_test_split(features_custom_concat, labels, test_size=0.2, random_state=6)

X_train_google_mean_tri, X_test_google_mean_tri, y_train_tri, y_test_tri = train_test_split(
    features_google_mean_tri, labels_tri, test_size=0.2, random_state=6
)
X_train_custom_mean_tri, X_test_custom_mean_tri, _, _ = train_test_split(
    features_custom_mean_tri, labels_tri, test_size=0.2, random_state=6
)
X_train_google_concat_tri, X_test_google_concat_tri, _, _ = train_test_split(
    features_google_concat_tri, labels_tri, test_size=0.2, random_state=6
)
X_train_custom_concat_tri, X_test_custom_concat_tri, _, _ = train_test_split(
    features_custom_concat_tri, labels_tri, test_size=0.2, random_state=6
)


train_dataset_google_mean = ReviewDataset(X_train_google_mean, y_train)
test_dataset_google_mean = ReviewDataset(X_test_google_mean, y_test)

train_dataset_custom_mean = ReviewDataset(X_train_custom_mean, y_train)
test_dataset_custom_mean = ReviewDataset(X_test_custom_mean, y_test)

train_dataset_google_concat = ReviewDataset(X_train_google_concat, y_train)
test_dataset_google_concat = ReviewDataset(X_test_google_concat, y_test)

train_dataset_custom_concat = ReviewDataset(X_train_custom_concat, y_train)
test_dataset_custom_concat = ReviewDataset(X_test_custom_concat, y_test)

train_dataset_google_mean_tri = ReviewDataset(X_train_google_mean_tri, y_train_tri)
test_dataset_google_mean_tri = ReviewDataset(X_test_google_mean_tri, y_test_tri)

train_dataset_custom_mean_tri = ReviewDataset(X_train_custom_mean_tri, y_train_tri)
test_dataset_custom_mean_tri = ReviewDataset(X_test_custom_mean_tri, y_test_tri)

train_dataset_google_concat_tri = ReviewDataset(X_train_google_concat_tri, y_train_tri)
test_dataset_google_concat_tri = ReviewDataset(X_test_google_concat_tri, y_test_tri)

train_dataset_custom_concat_tri = ReviewDataset(X_train_custom_concat_tri, y_train_tri)
test_dataset_custom_concat_tri = ReviewDataset(X_test_custom_concat_tri, y_test_tri)

train_loader_google_mean = DataLoader(train_dataset_google_mean, batch_size=64, shuffle=True)
test_loader_google_mean = DataLoader(test_dataset_google_mean, batch_size=64, shuffle=False)

train_loader_custom_mean = DataLoader(train_dataset_custom_mean, batch_size=64, shuffle=True)
test_loader_custom_mean = DataLoader(test_dataset_custom_mean, batch_size=64, shuffle=False)

train_loader_google_concat = DataLoader(train_dataset_google_concat, batch_size=64, shuffle=True)
test_loader_google_concat = DataLoader(test_dataset_google_concat, batch_size=64, shuffle=False)

train_loader_custom_concat = DataLoader(train_dataset_custom_concat, batch_size=64, shuffle=True)
test_loader_custom_concat = DataLoader(test_dataset_custom_concat, batch_size=64, shuffle=False)

train_loader_google_mean_tri = DataLoader(train_dataset_google_mean_tri, batch_size=64, shuffle=True)
test_loader_google_mean_tri = DataLoader(test_dataset_google_mean_tri, batch_size=64, shuffle=False)

train_loader_custom_mean_tri = DataLoader(train_dataset_custom_mean_tri, batch_size=64, shuffle=True)
test_loader_custom_mean_tri = DataLoader(test_dataset_custom_mean_tri, batch_size=64, shuffle=False)

train_loader_google_concat_tri = DataLoader(train_dataset_google_concat_tri, batch_size=64, shuffle=True)
test_loader_google_concat_tri = DataLoader(test_dataset_google_concat_tri, batch_size=64, shuffle=False)

train_loader_custom_concat_tri = DataLoader(train_dataset_custom_concat_tri, batch_size=64, shuffle=True)
test_loader_custom_concat_tri = DataLoader(test_dataset_custom_concat_tri, batch_size=64, shuffle=False)

print("Training MLP with Google Word2Vec Mean Features")
model_google_mean = SentimentClassifier(input_dim=300, num_classes=2)
train_model(model_google_mean, train_loader_google_mean, test_loader_google_mean)

print("Training MLP with Custom Word2Vec Mean Features")
model_custom_mean = SentimentClassifier(input_dim=300, num_classes=2)
train_model(model_custom_mean, train_loader_custom_mean, test_loader_custom_mean)

print("Training MLP with Google Word2Vec Concatenated Features")
model_google_concat = SentimentClassifier(input_dim=3000, num_classes=2)  # 10 * 300
train_model(model_google_concat, train_loader_google_concat, test_loader_google_concat)

print("Training MLP with Custom Word2Vec Concatenated Features")
model_custom_concat = SentimentClassifier(input_dim=3000, num_classes=2)  # 10 * 300
train_model(model_custom_concat, train_loader_custom_concat, test_loader_custom_concat)


print("Training MLP for Ternary Classification (Google Word2Vec Mean)")
model_google_mean_tri = SentimentClassifier(input_dim=300, num_classes=3)
train_model(model_google_mean_tri, train_loader_google_mean_tri, test_loader_google_mean_tri)

print("Training MLP for Ternary Classification (Custom Word2Vec Mean)")
model_custom_mean_tri = SentimentClassifier(input_dim=300, num_classes=3)
train_model(model_custom_mean_tri, train_loader_custom_mean_tri, test_loader_custom_mean_tri)

print("Training MLP for Ternary Classification (Google Word2Vec Concatenated)")
model_google_concat_tri = SentimentClassifier(input_dim=3000, num_classes=3)  # 10 * 300
train_model(model_google_concat_tri, train_loader_google_concat_tri, test_loader_google_concat_tri)

print("Training MLP for Ternary Classification (Custom Word2Vec Concatenated)")
model_custom_concat_tri = SentimentClassifier(input_dim=3000, num_classes=3)  # 10 * 300
train_model(model_custom_concat_tri, train_loader_custom_concat_tri, test_loader_custom_concat_tri)


Training MLP with Google Word2Vec Mean Features
Epoch 1, Loss: 62.5944, Train Accuracy: 0.6986
Epoch 2, Loss: 53.3788, Train Accuracy: 0.7852
Epoch 3, Loss: 50.7150, Train Accuracy: 0.8081
Epoch 4, Loss: 49.4936, Train Accuracy: 0.8177
Epoch 5, Loss: 48.7834, Train Accuracy: 0.8223
Epoch 6, Loss: 48.1755, Train Accuracy: 0.8300
Epoch 7, Loss: 47.5946, Train Accuracy: 0.8356
Epoch 8, Loss: 47.2357, Train Accuracy: 0.8394
Epoch 9, Loss: 46.7791, Train Accuracy: 0.8431
Epoch 10, Loss: 46.4788, Train Accuracy: 0.8442
Test Accuracy: 0.8225
Training MLP with Custom Word2Vec Mean Features
Epoch 1, Loss: 55.6917, Train Accuracy: 0.7534
Epoch 2, Loss: 48.3793, Train Accuracy: 0.8263
Epoch 3, Loss: 46.5380, Train Accuracy: 0.8477
Epoch 4, Loss: 45.3899, Train Accuracy: 0.8561
Epoch 5, Loss: 44.5267, Train Accuracy: 0.8658
Epoch 6, Loss: 43.7955, Train Accuracy: 0.8784
Epoch 7, Loss: 43.1144, Train Accuracy: 0.8833
Epoch 8, Loss: 42.7183, Train Accuracy: 0.8878
Epoch 9, Loss: 42.1645, Train Accur

SentimentClassifier(
  (fc1): Linear(in_features=3000, out_features=50, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

# Convolutional Neural Networks

In [7]:
MAX_LEN = 50 
class SentimentCNN(nn.Module):
    def __init__(self, input_dim=300, num_classes=2):
        super(SentimentCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        self.flattened_size = self._get_conv_output((1, input_dim, MAX_LEN))
        
        self.fc = nn.Linear(self.flattened_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def _get_conv_output(self, shape):
        with torch.no_grad():
            x = torch.zeros(shape)  
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            return x.numel()

    def forward(self, x):
        x = x.permute(0, 2, 1) 
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return self.softmax(x)


def compute_cnn_word2vec_custom(tokens, model, max_len=MAX_LEN):
    vectors = [model.wv[word] for word in tokens if word in model.wv][:max_len]
    while len(vectors) < max_len:
        vectors.append(np.zeros(model.vector_size))
    return np.array(vectors)
def compute_cnn_word2vec_google(tokens, model, max_len=MAX_LEN):
    vectors = [model[word] for word in tokens if word in model][:max_len]
    while len(vectors) < max_len:
        vectors.append(np.zeros(model.vector_size))
    return np.array(vectors)

all_reviews['w2v_google_cnn'] = all_reviews['token'].apply(lambda x: compute_cnn_word2vec_google(x, word2vec_model))
all_reviews['w2v_custom_cnn'] = all_reviews['token'].apply(lambda x: compute_cnn_word2vec_custom(x, custom_word2vec))

all_reviews_tri['w2v_google_cnn'] = all_reviews_tri['token'].apply(lambda x: compute_cnn_word2vec_google(x, word2vec_model))
all_reviews_tri['w2v_custom_cnn'] = all_reviews_tri['token'].apply(lambda x: compute_cnn_word2vec_custom(x, custom_word2vec))


features_google_cnn = np.stack(all_reviews['w2v_google_cnn'])
features_custom_cnn = np.stack(all_reviews['w2v_custom_cnn'])

features_google_cnn_tri = np.stack(all_reviews_tri['w2v_google_cnn'])
features_custom_cnn_tri = np.stack(all_reviews_tri['w2v_custom_cnn'])

labels = all_reviews['sentiment'].values 
labels_tri = all_reviews_tri['sentiment'].values 

X_train_google_cnn, X_test_google_cnn, y_train, y_test = train_test_split(
    features_google_cnn, labels, test_size=0.2, random_state=6
)
X_train_custom_cnn, X_test_custom_cnn, _, _ = train_test_split(
    features_custom_cnn, labels, test_size=0.2, random_state=6
)

X_train_google_cnn_tri, X_test_google_cnn_tri, y_train_tri, y_test_tri = train_test_split(
    features_google_cnn_tri, labels_tri, test_size=0.2, random_state=6
)
X_train_custom_cnn_tri, X_test_custom_cnn_tri, _, _ = train_test_split(
    features_custom_cnn_tri, labels_tri, test_size=0.2, random_state=6
)

train_dataset_google_cnn = ReviewDataset(X_train_google_cnn, y_train)
test_dataset_google_cnn = ReviewDataset(X_test_google_cnn, y_test)
train_dataset_custom_cnn = ReviewDataset(X_train_custom_cnn, y_train)
test_dataset_custom_cnn = ReviewDataset(X_test_custom_cnn, y_test)


train_dataset_google_cnn_tri = ReviewDataset(X_train_google_cnn_tri, y_train_tri)
test_dataset_google_cnn_tri = ReviewDataset(X_test_google_cnn_tri, y_test_tri)
train_dataset_custom_cnn_tri = ReviewDataset(X_train_custom_cnn_tri, y_train_tri)
test_dataset_custom_cnn_tri = ReviewDataset(X_test_custom_cnn_tri, y_test_tri)

train_loader_google_cnn = DataLoader(train_dataset_google_cnn, batch_size=64, shuffle=True)
test_loader_google_cnn = DataLoader(test_dataset_google_cnn, batch_size=64, shuffle=False)
train_loader_custom_cnn = DataLoader(train_dataset_custom_cnn, batch_size=64, shuffle=True)
test_loader_custom_cnn = DataLoader(test_dataset_custom_cnn, batch_size=64, shuffle=False)

train_loader_google_cnn_tri = DataLoader(train_dataset_google_cnn_tri, batch_size=64, shuffle=True)
test_loader_google_cnn_tri = DataLoader(test_dataset_google_cnn_tri, batch_size=64, shuffle=False)
train_loader_custom_cnn_tri = DataLoader(train_dataset_custom_cnn_tri, batch_size=64, shuffle=True)
test_loader_custom_cnn_tri = DataLoader(test_dataset_custom_cnn_tri, batch_size=64, shuffle=False)

print("Training CNN with Google Word2Vec")
model_google_cnn = SentimentCNN(input_dim=300, num_classes=2)  
train_model(model_google_cnn, train_loader_google_cnn, test_loader_google_cnn)

print("Training CNN with Custom Word2Vec")
model_custom_cnn = SentimentCNN(input_dim=300, num_classes=2)  
train_model(model_custom_cnn, train_loader_custom_cnn, test_loader_custom_cnn)

print("Training CNN for Ternary Classification (Google Word2Vec)")
model_google_cnn_tri = SentimentCNN(input_dim=300, num_classes=3) 
train_model(model_google_cnn_tri, train_loader_google_cnn_tri, test_loader_google_cnn_tri)

print("Training CNN for Ternary Classification (Custom Word2Vec)")
model_custom_cnn_tri = SentimentCNN(input_dim=300, num_classes=3) 
train_model(model_custom_cnn_tri, train_loader_custom_cnn_tri, test_loader_custom_cnn_tri)



Training CNN with Google Word2Vec
Epoch 1, Loss: 62.5583, Train Accuracy: 0.6603
Epoch 2, Loss: 51.9975, Train Accuracy: 0.7902
Epoch 3, Loss: 49.0883, Train Accuracy: 0.8197
Epoch 4, Loss: 46.9726, Train Accuracy: 0.8459
Epoch 5, Loss: 45.9028, Train Accuracy: 0.8575
Epoch 6, Loss: 44.9589, Train Accuracy: 0.8683
Epoch 7, Loss: 43.3237, Train Accuracy: 0.8841
Epoch 8, Loss: 42.0923, Train Accuracy: 0.8995
Epoch 9, Loss: 40.9464, Train Accuracy: 0.9095
Epoch 10, Loss: 39.8343, Train Accuracy: 0.9227
Test Accuracy: 0.8313
Training CNN with Custom Word2Vec
Epoch 1, Loss: 57.6043, Train Accuracy: 0.7325
Epoch 2, Loss: 47.6273, Train Accuracy: 0.8327
Epoch 3, Loss: 44.3549, Train Accuracy: 0.8683
Epoch 4, Loss: 42.4263, Train Accuracy: 0.8908
Epoch 5, Loss: 40.8278, Train Accuracy: 0.9084
Epoch 6, Loss: 39.2104, Train Accuracy: 0.9273
Epoch 7, Loss: 38.4052, Train Accuracy: 0.9347
Epoch 8, Loss: 37.6594, Train Accuracy: 0.9409
Epoch 9, Loss: 37.2303, Train Accuracy: 0.9437
Epoch 10, Loss: 

SentimentCNN(
  (conv1): Conv1d(300, 50, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(50, 10, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu): ReLU()
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=120, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)